In [3]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc

In [4]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    return features
    

In [10]:
audio_dir='Data/EXTRA'
label = 'REAL'
files = os.listdir(os.path.join(audio_dir,label))
for file in files:
    print(os.path.join(audio_dir,label,file))
    

Data/EXTRA/REAL/How to_femal.mp3
Data/EXTRA/REAL/How to Feel Energized & Sleep Better With One Morning Activity ｜ Dr. Andrew Huberman.mp3
Data/EXTRA/REAL/How to Rewrite Your Life ｜ Jully Black ｜ TEDxDownsviewWomen.mp3
Data/EXTRA/REAL/Maitriser l’intonation en français - phrase déclarative et interrogative.mp3
Data/EXTRA/REAL/MAITRISER L'INTONATION DE LA VOIX.mp3
Data/EXTRA/REAL/How to Invest New Cash： Dollar Cost Averaging vs. Lump Sum Investing.mp3
Data/EXTRA/REAL/How to Achieve Long Lasting Happiness.mp3
Data/EXTRA/REAL/How I Would Learn To Code (If I Could Start Over).mp3
Data/EXTRA/REAL/Describe Your Feelings in English ｜ Feelings and Emotions Vocabulary.mp3
Data/EXTRA/REAL/ONE GIRL — 17 ACCENTS.mp3


In [8]:
#make dataframe with all the features
def make_features_df(audio_dir):
    labels = ['REAL','FAKE']  # add 'FAKE' later # audio are in separate folders: FAKE and REAL
    feature_list = []
    for label in labels:
        files = os.listdir(os.path.join(audio_dir, label))
        for file in files:
            file_features=get_features(os.path.join(audio_dir,label,file),file)
            for segment_features in file_features:
                    feature_list.append(segment_features + [label])

    features_df = pd.DataFrame(feature_list,columns = data_set.columns)
    return features_df


In [11]:
data_set = pd.read_csv('Data/DATASET-balanced.csv')
extra_features = make_features_df(audio_dir)b

/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [12]:
extra_features

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.176064,0.087602,1418.507475,1296.095411,2738.876620,0.081721,-372.864349,46.615757,1.684505,4.311955,...,2.675245,-2.272016,0.175397,-1.240682,-1.662266,-4.981705,-2.533919,-4.527819,1.132139,REAL
1,0.237699,0.119661,1764.757184,2147.537816,3431.120162,0.061523,-224.734528,101.442604,-18.315481,2.435257,...,-1.218723,-14.079160,-7.538089,-8.501339,-8.854638,-5.643367,6.425860,-1.498871,8.455364,REAL
2,0.326273,0.074769,1849.606759,2027.537444,3671.166437,0.075806,-290.493683,109.520866,-20.313938,13.862265,...,13.581786,-5.443310,-2.681181,-6.856770,-2.239248,-7.944453,-10.046541,-6.950180,-0.450530,REAL
3,0.399162,0.052480,1674.614604,2050.047435,3585.278320,0.062433,-325.748505,116.308304,-15.593354,19.867882,...,5.328151,-5.407530,-6.249242,-6.328713,-6.071672,-2.833591,-6.080143,-9.470912,3.495361,REAL
4,0.308054,0.047366,1178.725592,1505.947913,1980.809992,0.053766,-363.053558,132.108414,1.911252,6.710779,...,1.618154,-3.712724,-6.065071,-8.155814,-11.147126,-9.586821,-4.618553,-6.935030,-0.104163,REAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5284,0.314406,0.046116,2698.440901,2535.992561,5354.182018,0.149414,-255.562592,87.417946,4.433177,13.824187,...,-0.741367,-4.572025,14.577326,-4.435467,-6.427833,4.626421,-0.856710,-12.452103,2.811413,REAL
5285,0.370378,0.056801,2955.994101,2328.057783,5294.231623,0.190552,-232.961044,76.125671,-1.108529,18.675051,...,2.659945,-9.495482,8.700753,-1.669142,-10.515453,0.587300,4.574445,-8.807771,4.212809,REAL
5286,0.309927,0.076453,1884.453339,2071.263641,3594.332053,0.091908,-237.585495,107.939301,-22.380022,18.590910,...,6.337625,-7.541246,11.250736,-8.606643,-7.723567,1.479188,-9.126733,-8.543749,0.512092,REAL
5287,0.385543,0.030032,2407.104292,2433.123626,4915.932395,0.120594,-269.907257,91.986984,9.712902,21.076706,...,2.552168,-6.178092,6.619003,-7.936121,-7.672117,0.772831,0.610292,-10.339297,0.709354,REAL


In [14]:
extra_features.to_csv('Data/Extra_features.csv',index=False)